In [5]:
import os
import requests
from chromadb import Client
from chromadb.config import Settings
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.getenv("EMBEDDING_KEY")
endpoint = os.getenv("EMBEDDING_END_POINT")

# Step 1: Test if environment variables are loaded correctly
if not api_key or not endpoint:
    print("Failed to load environment variables.")
    exit()
else:
    print("Environment variables loaded successfully.")

# Step 2: Initialize ChromaDB client
persist_directory = "chroma_data"
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

try:
    chroma_client = Client(Settings(persist_directory=persist_directory))
    print("ChromaDB initialized successfully.")
except Exception as e:
    print(f"Failed to initialize ChromaDB: {e}")
    exit()

# Step 3: Create a single embedding using the API
model_deployment = "text-embedding-ada-002"
api_version = "2024-04-01-preview"
url = f"{endpoint}/openai/deployments/{model_deployment}/embeddings?api-version={api_version}"

headers = {
    "api-key": api_key,
    "Content-Type": "application/json"
}

data_payload = {
    "input": "Test input for creating an embedding"
}

try:
    response = requests.post(url, headers=headers, json=data_payload)
    if response.status_code == 200:
        embedding = response.json()['data'][0]['embedding']
        print("Embedding created successfully.")
    else:
        print(f"Failed to create embedding: Status code {response.status_code}, Response: {response.text}")
        exit()
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")
    exit()

# Step 4: Store the embedding in ChromaDB
try:
    collection_name = "test_embeddings_collection"
    chroma_collection = chroma_client.get_or_create_collection(
        name=collection_name, 
        metadata={"description": "Test Collection"}
    )
    
    # Add the embedding to the collection
    chroma_collection.add(
        documents=["Test document for embedding storage."],
        metadatas=[{"description": "Test metadata"}],
        ids=["test_id_001"],
        embeddings=[embedding]
    )
    
    print("Embedding stored successfully in ChromaDB.")
    
    # Fetch the stored data to confirm persistence
    stored_data = chroma_collection.get()
    print("Stored Data:", stored_data)
    print("Embedding:", stored_data[0]['embedding'])

except Exception as e:
    print(f"Failed to store embedding in ChromaDB: {e}")


Environment variables loaded successfully.
ChromaDB initialized successfully.


Add of existing embedding ID: test_id_001
Insert of existing embedding ID: test_id_001


Embedding created successfully.
Embedding stored successfully in ChromaDB.
Stored Data: {'ids': ['test_id_001'], 'embeddings': None, 'documents': ['Test document for embedding storage.'], 'uris': None, 'data': None, 'metadatas': [{'description': 'Test metadata'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
Failed to store embedding in ChromaDB: 0
